In [9]:
# setup
%pip install litellm
%pip install pandas


[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
  Obtaining dependency information for pandas from https://files.pythonhosted.org/packages/52/11/9eac327a38834f162b8250aab32a6781339c69afe7574368fffe46387edf/pandas-2.2.3-cp311-cp311-macosx_11_0_arm64.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 6.3 MB/s eta 0:00:00
  Obtaining dependency information for numpy>=1.23.2 from https://files.pythonhosted.org/packages/47/7c/864cb966b96fce5e63fcf25e1e4d957fe5725a635e5f11fe03f39dd9d6b5/numpy-2.1.3-cp311-cp311-macosx_14_0_arm64.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 6.3 MB/s eta 0:00:00
  Obtaining dependency information for pytz>=2020.1 from https://files.pythonhosted.org/packages/11/c3/005fcca25ce078d2cc29fd559379817424e94885510568bc1bc53d7d5846/pytz-2024.2-py2.py3-none-any.whl.metadata
  Obtaining dependenc

In [10]:
# Insulin pump personal profile data
personal_profile_table = """
| Time Range   | Basal Rate (U/hr) | Correction Factor (1: mmol/L) | Carb Ratio (1: grams) | Target BG (mmol/L) |
|--------------|-------------------|------------------------------|-----------------------|---------------------|
| 12:00 AM     | 0.4               | 1:30                         | 1:10                  | 6.5                 |
| 2:00 AM      | 0.5               | 1:30                         | 1:10                  | 6.5                 |
| 5:00 AM      | 0.8               | 1:30                         | 1:10                  | 6.5                 |
| 6:00 AM      | 0.9               | 1:30                         | 1:10                  | 5.6                 |
| 7:00 AM      | 0.8               | 1:30                         | 1:10                  | 5.6                 |
| 8:00 AM      | 0.8               | 1:25                         | 1:10                  | 5.6                 |
| 9:00 AM      | 0.45              | 1:25                         | 1:10                  | 5.6                 |
| 12:00 PM     | 0.45              | 1:25                         | 1:10                  | 5.6                 |
| 1:00 PM      | 0.5               | 1:10                         | 1:10                  | 5.6                 |
| 3:00 PM      | 0.5               | 1:10                         | 1:10                  | 5.6                 |
| 4:00 PM      | 0.6               | 1:25                         | 1:10                  | 5.6                 |
| 7:00 PM      | 0.45              | 1:25                         | 1:12                  | 5.6                 |
| 9:00 PM      | 0.3               | 1:30                         | 1:10                  | 5.6                 |
| 11:00 PM     | 0.3               | 1:30                         | 1:10                  | 5.6                 |
"""

personal_profile_json = """
{
  "timed_settings": [
    {"time_range": "12:00 AM", "basal_rate": 0.4, "correction_factor": "1:30", "carb_ratio": "1:10", "target_bg": 6.5},
    {"time_range": "2:00 AM", "basal_rate": 0.5, "correction_factor": "1:30", "carb_ratio": "1:10", "target_bg": 6.5},
    {"time_range": "5:00 AM", "basal_rate": 0.8, "correction_factor": "1:30", "carb_ratio": "1:10", "target_bg": 6.5},
    {"time_range": "6:00 AM", "basal_rate": 0.9, "correction_factor": "1:30", "carb_ratio": "1:10", "target_bg": 5.6},
    {"time_range": "7:00 AM", "basal_rate": 0.8, "correction_factor": "1:30", "carb_ratio": "1:10", "target_bg": 5.6},
    {"time_range": "8:00 AM", "basal_rate": 0.8, "correction_factor": "1:25", "carb_ratio": "1:10", "target_bg": 5.6},
    {"time_range": "9:00 AM", "basal_rate": 0.45, "correction_factor": "1:25", "carb_ratio": "1:10", "target_bg": 5.6},
    {"time_range": "12:00 PM", "basal_rate": 0.45, "correction_factor": "1:25", "carb_ratio": "1:10", "target_bg": 5.6},
    {"time_range": "1:00 PM", "basal_rate": 0.5, "correction_factor": "1:10", "carb_ratio": "1:10", "target_bg": 5.6},
    {"time_range": "3:00 PM", "basal_rate": 0.5, "correction_factor": "1:10", "carb_ratio": "1:10", "target_bg": 5.6},
    {"time_range": "4:00 PM", "basal_rate": 0.6, "correction_factor": "1:25", "carb_ratio": "1:10", "target_bg": 5.6},
    {"time_range": "7:00 PM", "basal_rate": 0.45, "correction_factor": "1:25", "carb_ratio": "1:12", "target_bg": 5.6},
    {"time_range": "9:00 PM", "basal_rate": 0.3, "correction_factor": "1:30", "carb_ratio": "1:10", "target_bg": 5.6},
    {"time_range": "11:00 PM", "basal_rate": 0.3, "correction_factor": "1:30", "carb_ratio": "1:10", "target_bg": 5.6}
  ]
}
"""

personal_profile_xml = """
<insulin_pump_timed_settings>
  <setting>
    <time_range>12:00 AM</time_range>
    <basal_rate>0.4</basal_rate>
    <correction_factor>1:30</correction_factor>
    <carb_ratio>1:10</carb_ratio>
    <target_bg>6.5</target_bg>
  </setting>
  <setting>
    <time_range>2:00 AM</time_range>
    <basal_rate>0.5</basal_rate>
    <correction_factor>1:30</correction_factor>
    <carb_ratio>1:10</carb_ratio>
    <target_bg>6.5</target_bg>
  </setting>
  <setting>
    <time_range>5:00 AM</time_range>
    <basal_rate>0.8</basal_rate>
    <correction_factor>1:30</correction_factor>
    <carb_ratio>1:10</carb_ratio>
    <target_bg>6.5</target_bg>
  </setting>
  <setting>
    <time_range>6:00 AM</time_range>
    <basal_rate>0.9</basal_rate>
    <correction_factor>1:30</correction_factor>
    <carb_ratio>1:10</carb_ratio>
    <target_bg>5.6</target_bg>
  </setting>
  <setting>
    <time_range>7:00 AM</time_range>
    <basal_rate>0.8</basal_rate>
    <correction_factor>1:30</correction_factor>
    <carb_ratio>1:10</carb_ratio>
    <target_bg>5.6</target_bg>
  </setting>
  <setting>
    <time_range>8:00 AM</time_range>
    <basal_rate>0.8</basal_rate>
    <correction_factor>1:25</correction_factor>
    <carb_ratio>1:10</carb_ratio>
    <target_bg>5.6</target_bg>
  </setting>
  <setting>
    <time_range>9:00 AM</time_range>
    <basal_rate>0.45</basal_rate>
    <correction_factor>1:25</correction_factor>
    <carb_ratio>1:10</carb_ratio>
    <target_bg>5.6</target_bg>
  </setting>
  <setting>
    <time_range>12:00 PM</time_range>
    <basal_rate>0.45</basal_rate>
    <correction_factor>1:25</correction_factor>
    <carb_ratio>1:10</carb_ratio>
    <target_bg>5.6</target_bg>
  </setting>
  <setting>
    <time_range>1:00 PM</time_range>
    <basal_rate>0.5</basal_rate>
    <correction_factor>1:10</correction_factor>
    <carb_ratio>1:10</carb_ratio>
    <target_bg>5.6</target_bg>
  </setting>
  <setting>
    <time_range>3:00 PM</time_range>
    <basal_rate>0.5</basal_rate>
    <correction_factor>1:10</correction_factor>
    <carb_ratio>1:10</carb_ratio>
    <target_bg>5.6</target_bg>
  </setting>
  <setting>
    <time_range>4:00 PM</time_range>
    <basal_rate>0.6</basal_rate>
    <correction_factor>1:25</correction_factor>
    <carb_ratio>1:10</carb_ratio>
    <target_bg>5.6</target_bg>
  </setting>
  <setting>
    <time_range>7:00 PM</time_range>
    <basal_rate>0.45</basal_rate>
    <correction_factor>1:25</correction_factor>
    <carb_ratio>1:12</carb_ratio>
    <target_bg>5.6</target_bg>
  </setting>
  <setting>
    <time_range>9:00 PM</time_range>
    <basal_rate>0.3</basal_rate>
    <correction_factor>1:30</correction_factor>
    <carb_ratio>1:10</carb_ratio>
    <target_bg>5.6</target_bg>
  </setting>
  <setting>
    <time_range>11:00 PM</time_range>
    <basal_rate>0.3</basal_rate>
    <correction_factor>1:30</correction_factor>
    <carb_ratio>1:10</carb_ratio>
    <target_bg>5.6</target_bg>
  </setting>
</insulin_pump_timed_settings>
"""

In [43]:
# Integration with groq
from litellm import completion
import pandas as pd

system_message = (
    "You are a medical assistant that analyzes continuous glucose monitoring (CGM) data to provide personalized recommendations for insulin pump adjustments."
    "Based on glucose trends, make data-driven suggestions to optimize insulin delivery rates, focusing on preventing both hyperglycemia and hypoglycemia."
)

folder = "data_28_Oct-10_Nov_2024"
alarms_data_path = f'./{folder}/alarms_data_1.csv'
cgm_data_path = f'./{folder}/cgm_data_1.csv'
bolus_data_path = f'./{folder}/Insulin data/bolus_data_1.csv'  # Assuming this file is in "Insulin data" folder
insulin_per_day_data_path = f'./{folder}/Insulin data/insulin_data_1.csv'  # Assuming this file is in "Insulin data" folder
basal_data_path = f'./{folder}/Insulin data/basal_data_1.csv'  # Assuming this file is in "Insulin data" folder

alarms_data = pd.read_csv(alarms_data_path, skiprows=1)
cgm_data = pd.read_csv(cgm_data_path, skiprows=1)
bolus_data = pd.read_csv(bolus_data_path, skiprows=1)
insulin_per_day_data = pd.read_csv(insulin_per_day_data_path, skiprows=1)
# Not using basal_data because it states pretty stable and confuses the model
# basal_data = pd.read_csv(basal_data_path)

alarms_data = alarms_data.iloc[:, :-1] # remove Serial Number column
cgm_data = cgm_data.iloc[:, :-1] # remove Serial Number column
bolus_data = bolus_data.iloc[:, :-3] # remove last 3 (Initial Delivery (U),Extended Delivery (U),Serial Number) column
insulin_per_day_data = insulin_per_day_data.iloc[:, :-1] # remove Serial Number column
user_message = f"""

"""
print(alarms_data.head())
# print(cgm_data.head())
# print(bolus_data.head())
# print(insulin_per_day_data.head())

          Timestamp       Alarm/Event
0  2024-11-10 02:41  tandem_cgm_low_2
1  2024-11-09 20:51   tandem_cgm_high
2  2024-11-09 13:56  tandem_cgm_low_2
3  2024-11-09 13:36    tandem_cgm_low
4  2024-11-09 13:21  tandem_cgm_low_2


In [ ]:
response = completion(
    model="groq/llama-3.1-8b-instant", 
    messages=[
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_message}
   ],
)
print(response.choices[0].message.content)